<a href="https://colab.research.google.com/github/Tkaheva/MO/blob/main/homework_practice_5_1p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Метрики качества классификации ч.1

В уроке мы разбирали матрицу ошибок и некоторые метрики, основанные на терминах матрицы ошибок. Здесь же посмотрим, как эти метрики применять на практике и что  по ним можно выяснить.

### 1.1
Выкачайте тренировочный датасет Titanic - train.csv - с сайта [kaggle](https://www.kaggle.com/c/titanic/data). С помощью функции pd.read_csv() загрузите данные в датафрейм. Выведите первые 20 строк и проанализируйте данные: какие колонки присутствуют (более конкретная информация по ним есть на сайте kaggle), каким образом в них обозначены данные и какие типы данных используются (используйте pandas.dtypes).

In [4]:
# Сначала загрузим файл с Kaggle
import pandas as pd
import numpy as np
import io
import requests
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score, f1_score, classification_report

# Вариант 1: Если файл уже есть в Google Colab
try:
    df = pd.read_csv('train.csv')
    print("Файл найден локально")
except:
    print("Файл не найден локально. Загружаем с GitHub...")
    # Альтернативный вариант: загрузка с GitHub
    url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
    df = pd.read_csv(url)
    print("Файл загружен с GitHub")


Файл не найден локально. Загружаем с GitHub...
Файл загружен с GitHub


In [5]:
print("\n" + "="*50)
print("1.1 Анализ данных")
print("="*50)
print("Первые 20 строк:")
print(df.head(20))
print("\nИнформация о типах данных:")
print(df.dtypes)
print("\nРазмер датасета:", df.shape)
print("\nКолонки:", list(df.columns))



1.1 Анализ данных
Первые 20 строк:
    PassengerId  Survived  Pclass  \
0             1         0       3   
1             2         1       1   
2             3         1       3   
3             4         1       1   
4             5         0       3   
5             6         0       3   
6             7         0       1   
7             8         0       3   
8             9         1       3   
9            10         1       2   
10           11         1       3   
11           12         1       1   
12           13         0       3   
13           14         0       3   
14           15         0       3   
15           16         1       2   
16           17         0       3   
17           18         1       2   
18           19         0       3   
19           20         1       3   

                                                 Name     Sex   Age  SibSp  \
0                             Braund, Mr. Owen Harris    male  22.0      1   
1   Cumings, Mrs. John Bradley

### 1.2
Проверьте, имеются ли пропущенные значения в колонках, и выведите сумму всех пропущенных значений в каждой из колонок.

In [8]:

missing_values = df.isnull().sum()
print("Пропущенные значения по колонкам:")
print(missing_values[missing_values > 0])

Пропущенные значения по колонкам:
Age         177
Cabin       687
Embarked      2
dtype: int64


### 1.3
Замените все пропущенные значения колонки Age на медианы в зависимости от пола человека: т.е. если пол человека в строке с пропущенным значением "male", заменяете пропущенное значение возраста на медиану по всем известным возрастам мужчин, и наоборот. Выведите медианы возраста в зависимости от пола. Пропущенные значения колонок Cabin и Embarked замените на U (Unknown).

Примечание: для группировки по полу можно использовать метод df.groupby(), который имеет встроенные агрегатные функции (в т.ч. для вычисления медианы).  Для установки пропущенных значений в датасете воспользуйтесь функцией df.apply().

In [9]:
print("\n" + "="*50)
print("1.3 Заполнение пропущенных значений")
print("="*50)

# Медианы возраста по полу
male_median = df[df['Sex'] == 'male']['Age'].median()
female_median = df[df['Sex'] == 'female']['Age'].median()
print(f"Медиана возраста мужчин: {male_median}")
print(f"Медиана возраста женщин: {female_median}")

# Функция для заполнения возраста
def fill_age(row):
    if pd.isnull(row['Age']):
        if row['Sex'] == 'male':
            return male_median
        else:
            return female_median
    return row['Age']

df['Age'] = df.apply(fill_age, axis=1)

# Замена пропущенных значений в Cabin и Embarked
df['Cabin'] = df['Cabin'].fillna('U')
df['Embarked'] = df['Embarked'].fillna('U')

print("После заполнения пропусков:")
print(df.isnull().sum().sum(), "пропусков всего")



1.3 Заполнение пропущенных значений
Медиана возраста мужчин: 29.0
Медиана возраста женщин: 27.0
После заполнения пропусков:
0 пропусков всего


### 1.4
Выведите возраст пассажиров с PassengerID = [6, 20]. Убедитесь, что заполнены все пропущенные значения (воспользуйтесь функцией df.isnull() ).

In [10]:
print("\n" + "="*50)
print("1.4 Проверка конкретных пассажиров")
print("="*50)
print("Данные пассажиров с ID 6 и 20:")
print(df[df['PassengerId'].isin([6, 20])][['PassengerId', 'Age', 'Sex', 'Name']])
print("\nПроверка пропущенных значений после заполнения:")
print("Всего пропусков:", df.isnull().sum().sum())


1.4 Проверка конкретных пассажиров
Данные пассажиров с ID 6 и 20:
    PassengerId   Age     Sex                     Name
5             6  29.0    male         Moran, Mr. James
19           20  27.0  female  Masselmani, Mrs. Fatima

Проверка пропущенных значений после заполнения:
Всего пропусков: 0


Так как не все классификаторы могут работать с категориальными признаками, в данном случае необходимо заменить их на числовые.

Также можно было бы выделить расширенные признаки, такие как статус пассажира (Mr., Mrs., Miss., Dr., Master. и т.д.), собственный индекс билета, индекс палубы и другие, но здесь ограничимся выбором более простых признаков.

### 1.5
В колонке Sex замените значения на 0, если пол "male", и на 1, если "female". В колонке Embarked замените параметры "U", "S", "C", "Q" на 0, 1, 2, 3 соответственно. Отбросьте колонки PassengerId, Name, Ticket, Cabin. Выведите первые 20 строк получившегося набора данных.

In [11]:
print("\n" + "="*50)
print("1.5 Преобразование категориальных признаков")
print("="*50)

# Замена Sex на числовые значения
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# Замена Embarked на числовые значения
embarked_mapping = {'U': 0, 'S': 1, 'C': 2, 'Q': 3}
df['Embarked'] = df['Embarked'].map(embarked_mapping)

# Удаление ненужных колонок
df_processed = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
print("Первые 20 строк обработанного датасета:")
print(df_processed.head(20))
print("\nРазмер обработанного датасета:", df_processed.shape)


1.5 Преобразование категориальных признаков
Первые 20 строк обработанного датасета:
    Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0          0       3    0  22.0      1      0   7.2500         1
1          1       1    1  38.0      1      0  71.2833         2
2          1       3    1  26.0      0      0   7.9250         1
3          1       1    1  35.0      1      0  53.1000         1
4          0       3    0  35.0      0      0   8.0500         1
5          0       3    0  29.0      0      0   8.4583         3
6          0       1    0  54.0      0      0  51.8625         1
7          0       3    0   2.0      3      1  21.0750         1
8          1       3    1  27.0      0      2  11.1333         1
9          1       2    1  14.0      1      0  30.0708         2
10         1       3    1   4.0      1      1  16.7000         1
11         1       1    1  58.0      0      0  26.5500         1
12         0       3    0  20.0      0      0   8.0500         1
13   

### 1.6
Приступим к построению классификаторов. **Условимся, что, если функция или объект модели имеют параметр random_state, то устанавливаем его равным 17 в каждом из случаев.**

Разделите данные на тренировочный и тестовый датасеты, установив размер тестового как 0.25 (первая колонка Survived является целевой, поэтому необходимо сначала ее отделить от признаков).

In [14]:
from sklearn.model_selection import train_test_split

In [16]:
print("\n" + "="*50)
print("1.6 Разделение данных на train/test")
print("="*50)

X = df_processed.drop('Survived', axis=1)
y = df_processed['Survived']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=17, stratify=y
)

print(f"Размеры выборок:")
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")
print(f"Распределение классов в y_train: {y_train.value_counts().to_dict()}")
print(f"Распределение классов в y_test: {y_test.value_counts().to_dict()}")


1.6 Разделение данных на train/test
Размеры выборок:
X_train: (668, 7), X_test: (223, 7)
y_train: (668,), y_test: (223,)
Распределение классов в y_train: {0: 412, 1: 256}
Распределение классов в y_test: {0: 137, 1: 86}


### 1.7
Обучите на полученных выборках несколько классификаторов, которые импортированы ниже.

**Примечание: в методе ближайших соседей используйте количество соседей, равное 5.**

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [18]:
print("\n" + "="*50)
print("1.7 Обучение классификаторов")
print("="*50)

models = {
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(random_state=17),
    'Logistic Regression': LogisticRegression(random_state=17, max_iter=1000)
}

trained_models = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    trained_models[name] = model
    print(f"{name}: обучен")


1.7 Обучение классификаторов
KNN: обучен
Naive Bayes: обучен
Decision Tree: обучен
Logistic Regression: обучен


Очевидно, в этом случае использовать метрику accuracy, которую мы применяли до этого, не совсем правильно: она не даст верную оценку классификатору, потому как выживших явно меньше, чем погибших.

Вспомним про такие метрики как точность и полнота. Здесь они подойдут хорошо, так как у нас имеются положительный и отрицательный классы, и определить корректно один из них может оказаться более важным .

### 1.8
В каждом из пунктов следующего теста выберите, что важнее максимизировать: точность (precision) или полноту (recall). Выпишите ответы.

1. Вероятность того, что при определенной поломке самолета он сможет долететь до пункта назначения (1 - долетел, 0 - не долетел).
2. Предсказание, представляет ли человек опасность, по анализу психического состояния (1 - представляет опасность, 0 - не представляет опасности).
3. Предсказание ухода клиента (1 - клиент ушел, 0 - остался).
4. Выявление рака на основе медицинских показателей (1 - болен раком, 0 - здоров).
5. Предсказание летальности при наблюдаемой мутации (1 - выживание, 0 - летальный исход).
6. Определение важности происшествия для экстренных служб (1 - важно, 0 - неважно).
7. Окупятся ли вложения в бизнес (1 - окупятся, 0 - не окупятся).

Ваш ответ:

1. <br>
2. <br>
3. <br>
4. <br>
5. <br>
6. <br>
7. <br>

In [19]:
print("\n" + "="*50)
print("1.8 Ответы на вопросы (что важнее: precision или recall)")
print("="*50)
print("1. recall - важно не пропустить случаи, когда самолет может не долететь")
print("2. precision - важно минимизировать ложные обвинения")
print("3. recall - важно выявить всех уходящих клиентов")
print("4. recall - важно не пропустить ни одного случая рака")
print("5. recall - важно выявить все летальные случаи")
print("6. recall - важно не пропустить важные происшествия")
print("7. precision - важно не вкладывать в заведомо убыточный бизнес")


1.8 Ответы на вопросы (что важнее: precision или recall)
1. recall - важно не пропустить случаи, когда самолет может не долететь
2. precision - важно минимизировать ложные обвинения
3. recall - важно выявить всех уходящих клиентов
4. recall - важно не пропустить ни одного случая рака
5. recall - важно выявить все летальные случаи
6. recall - важно не пропустить важные происшествия
7. precision - важно не вкладывать в заведомо убыточный бизнес


### 1.9
Определите, что важнее использовать в случае датасета Titanic: полноту или точность. Объясните, почему.

### 1.10
Для каждого классификатора выведите матрицу ошибок и самостоятельно рассчитайте метрику recall (расчеты должны присутствовать). Проверьте расчеты, воспользовавшись встроенной метрикой recall_score.

In [20]:
from sklearn.metrics import confusion_matrix, recall_score

In [21]:
print("\n" + "="*50)
print("1.9 Для датасета Titanic важнее использовать recall")
print("="*50)
print("Обоснование: В данном контексте положительный класс - это выжившие (Survived = 1).")
print("Важнее не пропустить тех, кто мог бы выжить (максимизировать TP),")
print("чем минимизировать ложные предсказания выживания.")
print("На практике это означало бы приоритет спасения людей.")
print("\nПример: Лучше эвакуировать несколько лишних человек (FP),")
print("чем оставить на борту кого-то, кто мог бы выжить (FN).")


1.9 Для датасета Titanic важнее использовать recall
Обоснование: В данном контексте положительный класс - это выжившие (Survived = 1).
Важнее не пропустить тех, кто мог бы выжить (максимизировать TP),
чем минимизировать ложные предсказания выживания.
На практике это означало бы приоритет спасения людей.

Пример: Лучше эвакуировать несколько лишних человек (FP),
чем оставить на борту кого-то, кто мог бы выжить (FN).


In [22]:
print("\n" + "="*50)
print("1.10 Матрицы ошибок и метрика recall")
print("="*50)

for name, model in trained_models.items():
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    # Ручной расчет recall
    TP = cm[1, 1]  # True Positives
    FN = cm[1, 0]  # False Negatives
    manual_recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    # Расчет через sklearn
    sklearn_recall = recall_score(y_test, y_pred)

    print(f"\n{name}:")
    print(f"Матрица ошибок:\n{cm}")
    print(f"TN={cm[0,0]}, FP={cm[0,1]}, FN={cm[1,0]}, TP={cm[1,1]}")
    print(f"Ручной расчет recall = TP/(TP+FN) = {TP}/({TP}+{FN}) = {manual_recall:.4f}")
    print(f"Sklearn recall: {sklearn_recall:.4f}")
    print(f"Совпадают: {abs(manual_recall - sklearn_recall) < 0.0001}")


1.10 Матрицы ошибок и метрика recall

KNN:
Матрица ошибок:
[[111  26]
 [ 33  53]]
TN=111, FP=26, FN=33, TP=53
Ручной расчет recall = TP/(TP+FN) = 53/(53+33) = 0.6163
Sklearn recall: 0.6163
Совпадают: True

Naive Bayes:
Матрица ошибок:
[[116  21]
 [ 29  57]]
TN=116, FP=21, FN=29, TP=57
Ручной расчет recall = TP/(TP+FN) = 57/(57+29) = 0.6628
Sklearn recall: 0.6628
Совпадают: True

Decision Tree:
Матрица ошибок:
[[121  16]
 [ 31  55]]
TN=121, FP=16, FN=31, TP=55
Ручной расчет recall = TP/(TP+FN) = 55/(55+31) = 0.6395
Sklearn recall: 0.6395
Совпадают: True

Logistic Regression:
Матрица ошибок:
[[113  24]
 [ 29  57]]
TN=113, FP=24, FN=29, TP=57
Ручной расчет recall = TP/(TP+FN) = 57/(57+29) = 0.6628
Sklearn recall: 0.6628
Совпадают: True


### 1.11
Выберите ту модель, на которой метрика recall давала лучший ответ, и рассчитайте для нее precision, используя встроенную функцию.

In [ ]:
from sklearn.metrics import precision_score

In [23]:
print("\n" + "="*50)
print("1.11 Выбор лучшей модели по recall и расчет precision")
print("="*50)

best_model_name = None
best_recall = 0
recall_scores = {}

# Находим модель с максимальным recall
for name, model in trained_models.items():
    y_pred = model.predict(X_test)
    current_recall = recall_score(y_test, y_pred)
    recall_scores[name] = current_recall
    if current_recall > best_recall:
        best_recall = current_recall
        best_model_name = name

print("Recall для всех моделей:")
for name, score in recall_scores.items():
    print(f"{name}: {score:.4f}")

print(f"\nЛучшая модель по recall: {best_model_name}")
print(f"Recall лучшей модели: {best_recall:.4f}")

# Расчет precision для лучшей модели
best_model = trained_models[best_model_name]
y_pred_best = best_model.predict(X_test)
precision = precision_score(y_test, y_pred_best)

print(f"Precision лучшей модели: {precision:.4f}")

# Дополнительно выведем все метрики для лучшей модели
print(f"\nПолный отчет по классификации для {best_model_name}:")
print(classification_report(y_test, y_pred_best))
print(f"Accuracy: {accuracy_score(y_test, y_pred_best):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_best):.4f}")

# Вывод итоговой таблицы для всех моделей
print("\n" + "="*50)
print("Сводная таблица метрик для всех моделей")
print("="*50)
print(f"{'Модель':<20} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-score':<10}")
print("-" * 60)

for name, model in trained_models.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"{name:<20} {acc:<10.4f} {prec:<10.4f} {rec:<10.4f} {f1:<10.4f}")



1.11 Выбор лучшей модели по recall и расчет precision
Recall для всех моделей:
KNN: 0.6163
Naive Bayes: 0.6628
Decision Tree: 0.6395
Logistic Regression: 0.6628

Лучшая модель по recall: Naive Bayes
Recall лучшей модели: 0.6628
Precision лучшей модели: 0.7308

Полный отчет по классификации для Naive Bayes:
              precision    recall  f1-score   support

           0       0.80      0.85      0.82       137
           1       0.73      0.66      0.70        86

    accuracy                           0.78       223
   macro avg       0.77      0.75      0.76       223
weighted avg       0.77      0.78      0.77       223

Accuracy: 0.7758
F1-score: 0.6951

Сводная таблица метрик для всех моделей
Модель               Accuracy   Precision  Recall     F1-score  
------------------------------------------------------------
KNN                  0.7354     0.6709     0.6163     0.6424    
Naive Bayes          0.7758     0.7308     0.6628     0.6951    
Decision Tree        0.7892     0